In [ ]:
# === Cell 1: Unified Environment & Project-Wide Setup ===
import os, json, math, datetime as dt
from datetime import datetime  # for human-readable prints
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Optional (map preview): keep if you use geemap
try:
    import geemap
    GEEMAP_AVAILABLE = True
except Exception:
    GEEMAP_AVAILABLE = False

plt.style.use('seaborn-v0_8-whitegrid')
os.makedirs('outputs', exist_ok=True)

import ee

# --- Earth Engine authentication and initialization ---
# You can override with: export EE_PROJECT_ID=your-project-id
EE_PROJECT_ID = os.environ.get('EE_PROJECT_ID', 'nasa-flood')  # <— 원하는 기본 프로젝트 ID
def _ee_init():
    try:
        ee.Initialize()
        return "Initialized with default credentials"
    except Exception:
        try:
            ee.Initialize(project=EE_PROJECT_ID)
            return f"Initialized with project='{EE_PROJECT_ID}'"
        except Exception:
            print("Authenticating with Earth Engine...")
            ee.Authenticate()
            ee.Initialize(project=EE_PROJECT_ID)
            return f"Authenticated & initialized with project='{EE_PROJECT_ID}'"

print(_ee_init())
print(f"Current time: {datetime.now().isoformat(timespec='seconds')}")

# ===== Project-wide constants =====
CFG = {
    # AOIs (EPSG:4326)
    # Mekong Delta (VN lower basin); box chosen for stability/reproducibility
    "AOI_DELTA": ee.Geometry.Rectangle([104.30,  8.50, 106.90, 10.90], geodesic=False),
    # Tonlé Sap (KH); "Mekong's heartbeat"
    "AOI_TONLESAP": ee.Geometry.Rectangle([103.30, 12.00, 105.20, 13.70], geodesic=False),

    # Analysis windows
    "YEARS": list(range(2015, 2025)),
    "FLOOD_MONTHS": (8, 9),     # Aug–Sep (wet-season peak)
    "DROUGHT_MONTHS": (3, 4),   # Mar–Apr (dry-season trough)

    # Thresholds (physical/empirical, fixed for comparability)
    "TH_VV_DB": -16.0,
    "TH_VH_DB": -22.0,

    # Landsat5 baseline window (pre-major-dam reference)
    "BASELINE_YEARS": [2005, 2006, 2007, 2008],

    # Event markers for plots
    "EVENTS": {
        "JINGHONG_FLOW_CUT": "2019-07-15",  # Jinghong flow cut (smoking gun)
        "XIAOWAN_ONLINE":    "2009-01-01",
        "NUOZHADU_ONLINE":   "2012-01-01"
    }
}

# ===== Utilities for Earth Engine =====
def _daterange_of_year_months(year:int, m1:int, m2:int):
    """Return ISO start and inclusive end-of-month last day for [m1..m2]."""
    start = dt.date(year, m1, 1)
    if m2 == 12:
        end = dt.date(year+1, 1, 1) - dt.timedelta(days=1)
    else:
        end = dt.date(year, m2+1, 1) - dt.timedelta(days=1)
    return start.isoformat(), end.isoformat()

def s1_min(aoi, start, end, pol):
    """Min-composite Sentinel-1 GRD over period to stabilize speckle."""
    return (ee.ImageCollection('COPERNICUS/S1_GRD')
            .filterBounds(aoi)
            .filterDate(start, end)
            .filter(ee.Filter.eq('instrumentMode','IW'))
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', pol))
            .select(pol)
            .min()
            .clip(aoi))

def classify_water(img_min, pol, threshold_db):
    """Water = backscatter < threshold (in dB); returns self-masked binary."""
    return img_min.lt(threshold_db).selfMask()

def area_km2(mask_img, aoi, scale=30, band_name='constant'):
    """Compute km² of a self-masked image."""
    area = (mask_img.multiply(ee.Image.pixelArea())
            .reduceRegion(ee.Reducer.sum(), aoi, scale, maxPixels=1e12))
    # selfMask() produces band 'constant'
    return ee.Number(area.get(band_name)).divide(1e6)

def landsat5_c2_sr_mask_scale(img):
    """L5 C2 L2 scaling + cloud/shadow masking."""
    qa = img.select('QA_PIXEL')
    cloud  = 1 << 3
    shadow = 1 << 4
    mask = qa.bitwiseAnd(cloud).eq(0).And(qa.bitwiseAnd(shadow).eq(0))
    optical = img.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal = img.select('ST_B6').multiply(0.00341802).add(149.0)
    return (img.addBands(optical, None, True)
               .addBands(thermal, None, True)
               .updateMask(mask))

def landsat5_median(aoi, start_date, end_date, months=None):
    """Median composite with optional wrapped-month filter ((11,12),(1,4))."""
    col = (ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
           .filterBounds(aoi)
           .filterDate(start_date, end_date)
           .map(landsat5_c2_sr_mask_scale))
    if months:
        # e.g., months=((11,12),(1,4)) to handle dry-season wrap
        f = ee.Filter.Or(
            ee.Filter.calendarRange(int(months[0][0]), int(months[0][1]), 'month'),
            ee.Filter.calendarRange(int(months[1][0]), int(months[1][1]), 'month')
        )
        col = col.filter(f)
    return col.median().clip(aoi)

def mndwi(img):  # SR_B2 (Green), SR_B5 (SWIR)
    return img.normalizedDifference(['SR_B2','SR_B5']).rename('MNDWI')

def water_mask_from_mndwi(img, threshold=0.0):
    return img.gt(threshold).selfMask()

def chirps_sum_mm(aoi, start, end):
    """Return AOI-mean of CHIRPS precipitation sum (mm) over [start,end]."""
    col = (ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
           .filterBounds(aoi).filterDate(start, end).select('precipitation'))
    total = col.sum().reduceRegion(ee.Reducer.mean(), aoi, 5000, maxPixels=1e12)
    return ee.Number(total.get('precipitation'))

# (Optional) quick summary print
print(f"AOI_DELTA bounds: {CFG['AOI_DELTA'].bounds().getInfo()['coordinates'][0][0]} …")
print(f"AOI_TONLESAP bounds: {CFG['AOI_TONLESAP'].bounds().getInfo()['coordinates'][0][0]} …")
print("Setup complete ✅")


In [ ]:
# === Cell 2: Parameters & ancillary datasets (WorldCover, NASADEM) ===
# 스무딩(모폴로지) 파라미터: 소금후추 노이즈 억제
SMOOTH_RADIUS_M = 30   # 30 m (약 3픽셀); 상황에 따라 20~50 m 튜닝 가능

# 경사도(나사DEM) 마스크: 급경사 지역의 오탐 억제
SLOPE_DEG_MAX = 5      # 홍수 후보지: 대개 0~5도 평지

# 분석 윈도우: 기본은 FLOOD_MONTHS (08~09)
FLOOD_M1, FLOOD_M2 = CFG["FLOOD_MONTHS"]

# Ancillary: NASADEM slope
dem = ee.Image('USGS/NASADEM_HGT')
slope_deg = ee.Terrain.slope(dem)
flat_mask = slope_deg.lte(SLOPE_DEG_MAX)  # 평지 마스크

# Ancillary: WorldCover (cropland=40)
# v200도 있지만, 경로/연도 바뀌면 오류가 잦으므로 안정적인 v100/2020 고정
worldcover = ee.Image('ESA/WorldCover/v100/2020').select('Map')
cropland_mask = worldcover.eq(40).selfMask()


In [ ]:
# === Cell 3: Morphology helpers & dual-pol flood mask ===
def morph_open(img, radius_m=SMOOTH_RADIUS_M):
    # 작은 점 광고 제거 (erosion -> dilation)
    return (img.focal_min(radius_m, 'circle', 'meters')
               .focal_max(radius_m, 'circle', 'meters'))

def morph_close(img, radius_m=SMOOTH_RADIUS_M):
    # 작은 구멍 메움 (dilation -> erosion)
    return (img.focal_max(radius_m, 'circle', 'meters')
               .focal_min(radius_m, 'circle', 'meters'))

def refine_binary(mask_img):
    # open으로 점 제거 후 close로 구멍 메움
    m1 = morph_open(mask_img)
    m2 = morph_close(m1)
    return m2

def dualpol_masks_for_period(aoi, start_iso, end_iso, th_vv, th_vh):
    """주어진 기간에 대해 VV(open water), VH(flooded veg 포함) 마스크와 VH-only(차집합) 반환"""
    vv_min = s1_min(aoi, start_iso, end_iso, 'VV')
    vh_min = s1_min(aoi, start_iso, end_iso, 'VH')

    vv_raw = classify_water(vv_min, 'VV', th_vv)        # 개방수역
    vh_raw = classify_water(vh_min, 'VH', th_vh)        # 개방수역+식생하 침수 포함

    # 스무딩 & 경사도(평지) 제한
    vv_mask = refine_binary(vv_raw).updateMask(flat_mask)
    vh_mask = refine_binary(vh_raw).updateMask(flat_mask)

    # VH-only = VH - VV
    vh_only = vh_mask.subtract(vv_mask.unmask(0)).selfMask()
    return vv_mask, vh_mask, vh_only


In [ ]:
# === Cell 4: Yearly stats (Aug–Sep) per AOI, inc. cropland impact ===
def period_iso_for_year(year:int, m1:int=FLOOD_M1, m2:int=FLOOD_M2):
    start, end = _daterange_of_year_months(year, m1, m2)
    return start, end

def stats_for_year_aoi(year:int, aoi, aoi_label:str, th_vv, th_vh):
    start, end = period_iso_for_year(year)
    vv_mask, vh_mask, vh_only = dualpol_masks_for_period(aoi, start, end, th_vv, th_vh)

    # 전체 면적(km²)
    vv_km2      = float(area_km2(vv_mask, aoi, 30).getInfo() or 0.0)
    vh_km2      = float(area_km2(vh_mask, aoi, 30).getInfo() or 0.0)
    vh_only_km2 = float(area_km2(vh_only, aoi, 30).getInfo() or 0.0)

    # 농경지 피해 (VH-only ∩ cropland)
    flooded_cropland = vh_only.updateMask(cropland_mask)
    flooded_cropland_km2 = float(area_km2(flooded_cropland, aoi, 30).getInfo() or 0.0)

    # 미탐률(= VV만 썼을 때 놓치는 비율)
    missed_pct = (vh_only_km2 / vh_km2 * 100.0) if vh_km2 > 0 else 0.0

    return {
        "year": year, "aoi": aoi_label,
        "vv_km2": vv_km2,
        "vh_km2": vh_km2,
        "vh_only_km2": vh_only_km2,
        "missed_by_vv_pct": missed_pct,
        "cropland_vhonly_km2": flooded_cropland_km2
    }

rows = []
for y in CFG["YEARS"]:
    rows.append(stats_for_year_aoi(y, CFG["AOI_DELTA"],    "Mekong_Delta", CFG["TH_VV_DB"], CFG["TH_VH_DB"]))
    rows.append(stats_for_year_aoi(y, CFG["AOI_TONLESAP"], "Tonle_Sap",    CFG["TH_VV_DB"], CFG["TH_VH_DB"]))

df_dual = pd.DataFrame(rows).sort_values(["aoi","year"]).reset_index(drop=True)
display(df_dual.head(6))
print(df_dual.shape)

# 저장
out_csv = "outputs/dualpol_flood_stats_2015_2024.csv"
df_dual.to_csv(out_csv, index=False)
print("Saved ->", out_csv)


In [ ]:
# === Cell 5: Visualization (stacked bars + missed %) ===
from matplotlib.ticker import FuncFormatter

def plot_stack_for_aoi(df, aoi_label, fname_png):
    sub = df[df["aoi"]==aoi_label].copy()

    x = sub["year"].values
    vv = sub["vv_km2"].values
    vh_only = sub["vh_only_km2"].values
    missed = sub["missed_by_vv_pct"].values

    fig, ax = plt.subplots(figsize=(12,6))
    ax.bar(x, vv, label="Open water (VV)", width=0.7)
    ax.bar(x, vh_only, bottom=vv, label="Flooded vegetation (VH-only)", width=0.7)

    # 각 연도 위에 미탐율(%) 라벨
    for xi, v_base, v_add, pct in zip(x, vv, vh_only, missed):
        y = v_base + v_add
        ax.text(xi, y + max(1, y*0.01), f"Missed {pct:.1f}%", ha='center', va='bottom', fontsize=9)

    ax.set_title(f"{aoi_label} — Dual-Pol Flood (Aug–Sep), VV vs VH-only")
    ax.set_xlabel("Year"); ax.set_ylabel("Flood extent (km²)")
    ax.yaxis.set_major_formatter(FuncFormatter(lambda v, p: f"{int(v):,}"))
    ax.legend()
    ax.grid(True, axis='y')
    plt.tight_layout()
    plt.savefig(fname_png, dpi=200)
    plt.show()
    print("Saved ->", fname_png)

plot_stack_for_aoi(df_dual, "Mekong_Delta", "outputs/dualpol_stack_delta.png")
plot_stack_for_aoi(df_dual, "Tonle_Sap",    "outputs/dualpol_stack_tonlesap.png")


In [ ]:
# === Cell 6: Visualization (cropland impact) ===
def plot_cropland_impact(df, aoi_label, fname_png):
    sub = df[df["aoi"]==aoi_label].copy()

    fig, ax = plt.subplots(figsize=(12,6))
    ax.plot(sub["year"], sub["cropland_vhonly_km2"], marker='o', linestyle='-')
    ax.set_title(f"{aoi_label} — Flooded cropland (VH-only ∩ WorldCover=40)")
    ax.set_xlabel("Year"); ax.set_ylabel("Area (km²)")
    ax.yaxis.set_major_formatter(lambda v, p: f"{int(v):,}")
    ax.grid(True, axis='y')
    plt.tight_layout()
    plt.savefig(fname_png, dpi=200)
    plt.show()
    print("Saved ->", fname_png)

plot_cropland_impact(df_dual, "Mekong_Delta", "outputs/cropland_flood_delta.png")
plot_cropland_impact(df_dual, "Tonle_Sap",    "outputs/cropland_flood_tonlesap.png")


In [ ]:
# === Cell 7 (optional): Quick map for a chosen year & AOI ===
import geemap

def preview_map(aoi, center_xy, year, label_prefix):
    start, end = period_iso_for_year(year)
    vv_mask, vh_mask, vh_only = dualpol_masks_for_period(aoi, start, end, CFG["TH_VV_DB"], CFG["TH_VH_DB"])

    flooded_cropland = vh_only.updateMask(cropland_mask)

    m = geemap.Map(center=center_xy, zoom=8)
    m.addLayer(vv_mask, {'palette':'add8e6'}, f"{label_prefix} {year} Open water (VV)")
    m.addLayer(vh_only, {'palette':'00008b'}, f"{label_prefix} {year} Flooded veg (VH-only)")
    m.addLayer(flooded_cropland, {'palette':'ff8800'}, f"{label_prefix} {year} Cropland ∩ VH-only")
    m.addLayerControl()
    return m

# 예시: 2018년 델타 / 2019년 톤레삽
map_delta_2018 = preview_map(CFG["AOI_DELTA"], [9.9,105.7], 2018, "Delta")
map_ts_2019    = preview_map(CFG["AOI_TONLESAP"], [12.8,104.2], 2019, "Tonle Sap")
map_delta_2018, map_ts_2019


In [ ]:
# === Cell 8: Programmatic summary ===
def pct(v):
    return f"{v:.1f}%"

sum_delta = df_dual[df_dual["aoi"]=="Mekong_Delta"].copy()
sum_ts    = df_dual[df_dual["aoi"]=="Tonle_Sap"].copy()

summary_txt = f"""
[Note05 Summary — Dual-Polarization & Impact Quantification]
• VV(개방수역) + VH-only(식생하 침수) 분해를 통해 'VV만 쓸 때 놓치는 침수'를 정량화.
• 경사도(NASADEM ≤ {SLOPE_DEG_MAX}°) + 모폴로지 스무딩(±{SMOOTH_RADIUS_M} m)으로 오탐 억제 및 경계 안정화.
• WorldCover(cropland=40)와 교차하여 농경지 피해 면적(km²) 산출.

Mekong Delta (2015–2024):
- 평균 미탐률(VV 기준): {pct(sum_delta['missed_by_vv_pct'].mean())}
- 연최대 농경지 침수(VH-only∩cropland): {sum_delta['cropland_vhonly_km2'].max():,.0f} km²

Tonlé Sap (2015–2024):
- 평균 미탐률(VV 기준): {pct(sum_ts['missed_by_vv_pct'].mean())}
- 연최대 농경지 침수(VH-only∩cropland): {sum_ts['cropland_vhonly_km2'].max():,.0f} km²

Artifacts:
• outputs/dualpol_flood_stats_2015_2024.csv
• outputs/dualpol_stack_delta.png
• outputs/dualpol_stack_tonlesap.png
• outputs/cropland_flood_delta.png
• outputs/cropland_flood_tonlesap.png
"""

print(summary_txt)
with open("outputs/note05_summary.txt","w",encoding="utf-8") as f:
    f.write(summary_txt)
print("Saved -> outputs/note05_summary.txt")
